In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

train_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv',index_col='Id')
test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv',index_col='Id')

train_data.head()

In [ ]:
train_data.columns

In [ ]:
train_col_null = train_data.columns[train_data.isnull().any() == True].tolist()
train_data[train_col_null].isnull().sum()

In [ ]:
test_data.head()

In [ ]:
test_data.columns

In [ ]:
test_col_null = test_data.columns[test_data.isnull().any() == True].tolist()
test_data[test_col_null].isnull().sum()

In [ ]:
X = train_data.dropna(axis=0, subset=['SalePrice'])
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == 'object']

low_cardinality_cols

In [ ]:
numeric_col = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

numeric_col

In [ ]:
my_cols = low_cardinality_cols + numeric_col
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = test_data[my_cols].copy()
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)
xgb = XGBRegressor(n_estimators=2000,
                   learning_rate=0.05)
xgb.fit(X_train, y_train)

In [ ]:
y_pred = xgb.predict(X_valid)

prediction = xgb.predict(X_test)
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice':prediction})
output.to_csv('submission.csv', index=False)
output.head()